In [ ]:
#pendulum swing up 

using Pkg
Pkg.activate(".")
using LinearAlgebra
using ForwardDiff
using Plots 
using TSSOS 
using DynamicPolynomials
using DifferentialEquations  
using BlockDiagonals 

In [ ]:
#Pendulum parameters

#pendelum length (m)
l = 1.0

#mass of pendelum kg (assuming massless rod)
m = 1.0

#timestep (s) 
h = 0.1 #10 Hz

#h = 0.01 #100 Hz

#define gravity

g = [0.0; -9.81]

In [ ]:
#generate a feasible initial condition
#      ^ y
#______|__________->x
#      |\
#      |θ\
#      |  \
#      |   O

#initial angle 
θ_initial = pi/4

#initial cartesian position based off this angle
x_initial = l*sin(θ_initial)
y_initial = -l*cos(θ_initial)


#closer to the upright goal (working with this initial condition)
#x_initial = l*sin(θ_initial)
#y_initial = l*cos(θ_initial)


#initial velocity 
vx_initial = 0
vy_initial = 0


#initialize as 0...
λ_initial = 0

In [ ]:
#horizon
#3 seconds when running at 10 Hz
#seems to be working
#N = 30

#5 seconds
N = 50

In [ ]:
#gravity vector 
g = [0; -9.81] 

In [ ]:
#state is x = [qx, qy, vx, vy]

#control is u = [Fx, Fy]

In [ ]:
#this is the state trajectory we are solving for 
@polyvar x[1:4, 1:N]

#this is the control trajectory 
@polyvar u[1:2, 1:N-1]

#this is the slack variable taking care of the constraint 
#there are actually N-1 lambdas, think of it as a control
#initialization does not matter
@polyvar λ[1:1:N-1] 

In [ ]:
#state weighting matrix 
Q = 1000*Matrix(1.0*I, 4,4)

#controls weighting matrix 
R = 1*Matrix(1.0*I, 2,2)

#terminal cost weighting matrix
Qf = 1e6*Matrix(1.0*I, 4,4)

In [ ]:
#goal position is upright 

#      0
#      |
#      | y
#______|__________->x
#      |        

xgoal = [0.0, l , 0.0, 0.0]

In [ ]:
#cost function (quadratic LQR cost)
f = 0
for i = 1:N-1

    f += (x[:, i] - xgoal)'*Q*(x[:, i] - xgoal) + u[:, i]'*R*u[:, i]

end

#terminal cost 
f+=x[:, N]'*Qf*x[:, N]

In [ ]:
#initialize equality and inequality constraints 
eq = []
ineq = []

In [ ]:
#set the initial condition constraints
append!(eq, [x[1,1] - x_initial])
append!(eq, [x[2,1] - y_initial])
append!(eq, [x[3,1] - vx_initial])
append!(eq, [x[4,1] - vy_initial])

In [ ]:
#set the dynamics constraints between each timestep 

for i=1:N-1

    #implicit euler integration on position
    append!(eq, x[1:2, i]+h*x[3:4, i+1]-x[1:2, i+1])

    #implicit velocity constraints 
    #lambda k is between the two timesteps (technically the same as i+1)
    append!(eq, x[3:4, i] + h*(g+(2/m)*λ[i]*x[1:2, i+1] + (u[:,i]/m)) - x[3:4, i+1])

    #length constraint
    #starts at i+1 because we assume we give it a feasible initial condition 
    append!(eq, [x[1:2, i+1]'*x[1:2, i+1] - l^2])

end

In [ ]:
#add in control constraints 
for i=1:N-1

    #control in x and y less than or equal to 10
    append!(ineq, [10-u[1,i]])
    append!(ineq, [10-u[2,i]])

end

In [ ]:
pop = append!([f], ineq)

In [ ]:
pop = append!(pop, eq)

In [ ]:
#relaxation order
d = 2

In [ ]:
#concatenate all the variables together 
var = [vec(x); vec(u); λ]

In [ ]:
#solve 

#no correlative sparsity (runs slower)
#opt, sol, data = tssos_first(pop, var, d, numeq=length(eq), TS="MD")


#extract the moment matrix without ipopt solve to check global optimality (with chordal sparsity and term sparsity)
#opt, sol, data = cs_tssos_first(pop, var, d, numeq=length(eq), TS="MD", solution=false, Mommat=true)

#solution = true, extracts an approximately optimal solution
opt, sol, data = cs_tssos_first(pop, var, d, numeq=length(eq), TS="MD", solution=true)

In [ ]:
#0 if globally optimal 
data.flag 

In [ ]:
#these are the block diagonals of the moment matrix 
data.moment

In [ ]:
#first block 
data.moment[1]

In [ ]:
data.moment 

In [ ]:
#need to convert the data type to use this function
#mom = BlockDiagonal(Matrix(data.moment)) 

In [ ]:
all_condition_numbers = zeros(size(data.moment)[1]) 

for i=1:size(data.moment)[1]

    all_condition_numbers[i] = cond(data.moment[i])
    
end

In [ ]:
all_condition_numbers 

In [ ]:
#construct moment matrix

#initialize row size and column size 
row_size = 0
column_size = 0

#get the amount of rows 
for i=1:size(data.moment)[1]

    row_size += size(data.moment[i])[1]
    column_size += size(data.moment[i])[2]

end

#get the amount of columns

In [ ]:
#construct the moment matrix 
moment_matrix = zeros(row_size, column_size)

#initialize row and column index 
row_index = 1
column_index = 1

for i=1:size(data.moment)[1]

    moment_matrix[row_index:row_index + size(data.moment[i])[1]-1, column_index:column_index + size(data.moment[i])[2]-1] = data.moment[i]

    row_index += size(data.moment[i])[1]
    column_index += size(data.moment[i])[2]


end

In [ ]:
rank(moment_matrix) 

In [ ]:
cond(moment_matrix)

In [ ]:
size(data.moment[100])

In [ ]:
data.basis[1]

In [ ]:
findmax(all_condition_numbers)  

In [ ]:
sol 

In [ ]:
#need to reshape this 
x_traj = sol[1: size(vec(x))[1]]

x_traj = reshape(x_traj, 4, N)

In [ ]:
u_traj = sol[size(vec(x))[1]+1: size(vec(x))[1] + size(vec(u))[1]]

u_traj = reshape(u_traj, 2, N-1)

In [ ]:
λ_traj = sol[size(vec(x))[1] + size(vec(u))[1]+1: end]

In [ ]:
plot(x_traj[1,:], label=false, title="X Position")

In [ ]:
plot(x_traj[2,:], title="Y Position")

In [ ]:
plot(x_traj[3,:], title="X Velocity")

In [ ]:
plot(x_traj[4,:], title="Y Velocity")

In [ ]:
plot(u_traj[1,:], title="Controls", label="Fx")
plot!(u_traj[2,:], label="Fy")

In [ ]:
#check the constraints 

In [ ]:
constraint_check = zeros(5, N-1)

In [ ]:
for i=1:N-1

    constraint_check[1:2, i] = x_traj[1:2, i]+h*x_traj[3:4, i+1]-x_traj[1:2, i+1]

    constraint_check[3:4, i] = x_traj[3:4, i] + h*(g+(2/m)*λ_traj[i]*x_traj[1:2, i+1] + (u_traj[:,i]/m)) - x_traj[3:4, i+1]

    constraint_check[5, i] = x_traj[1:2, i+1]'*x_traj[1:2, i+1] - l^2
end

In [ ]:
constraint_check 

In [ ]:
#trajectory is dynamically feasible
plot(constraint_check') 

In [ ]:
#working 